# Chapter 4.3

## Loop Statements

SystemVerilog provides several loop constructs for different use cases.

## for Loop

The `for` loop is used when the number of iterations is known.

```systemverilog
for (initialization; condition; increment) begin
    // statements
end
```

### Example 6: Parallel-to-Serial Converter

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_6__parallel_to_serial/"
files = ["parallel_to_serial.sv", "parallel_to_serial_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// parallel_to_serial.sv
module parallel_to_serial(
    input logic clk, rst_n, load,
    input logic [7:0] parallel_in,
    output logic serial_out, done
);
    logic [7:0] shift_reg;
    logic [2:0] count;
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            shift_reg <= 8'h00;
            count <= 3'd0;
        end else if (load) begin
            shift_reg <= parallel_in;
            count <= 3'd0;
        end else if (count < 3'd7) begin
            shift_reg <= {shift_reg[6:0], 1'b0};
            count <= count + 1'b1;
        end
    end
    
    assign serial_out = shift_reg[7];
    assign done = (count == 3'd7);
endmodule
```

```systemverilog
// parallel_to_serial_testbench.sv
module parallel_to_serial_testbench;
    // Testbench signals
    logic clk, rst_n, load;
    logic [7:0] parallel_in;
    logic serial_out, done;
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10ns period clock
    end
    
    // Instantiate the design under test
    parallel_to_serial DUT (
        .clk(clk),
        .rst_n(rst_n),
        .load(load),
        .parallel_in(parallel_in),
        .serial_out(serial_out),
        .done(done)
    );
    
    // Task to display current state
    task display_state(string description);
        $display("%s", description);
        $display("Time: %4t | Load: %b | Parallel_in: %8b (%02h) | Serial_out: %b | Done: %b | Count: %d | Shift_reg: %8b", 
                 $time, load, parallel_in, parallel_in, serial_out, done, DUT.count, DUT.shift_reg);
        $display("---------------------------------------------------------");
    endtask
    
    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("parallel_to_serial_testbench.vcd");
        $dumpvars(0, parallel_to_serial_testbench);
        
        $display("Starting Parallel-to-Serial Converter Test");
        $display("==========================================");
        $display();
        
        // Initialize signals
        rst_n = 0;
        load = 0;
        parallel_in = 8'h00;
        
        // Reset test
        #10;
        display_state("After Reset:");
        
        // Release reset
        rst_n = 1;
        #10;
        display_state("Reset Released:");
        
        // Test 1: Load pattern 10101010 (0xAA)
        $display("\n=== TEST 1: Converting 0xAA (10101010) ===");
        parallel_in = 8'hAA;
        load = 1;
        #10;
        display_state("Data Loaded:");
        
        load = 0;
        
        // Shift out all 8 bits
        for (int i = 0; i < 8; i++) begin
            #10;
            $display("Cycle %d: Serial_out = %b, Done = %b, Count = %d, Shift_reg = %8b", 
                     i+1, serial_out, done, DUT.count, DUT.shift_reg);
        end
        
        #10;
        display_state("After all bits shifted:");
        
        // Test 2: Load pattern 11110000 (0xF0)
        $display("\n=== TEST 2: Converting 0xF0 (11110000) ===");
        parallel_in = 8'hF0;
        load = 1;
        #10;
        display_state("Data Loaded:");
        
        load = 0;
        
        // Shift out all 8 bits
        for (int i = 0; i < 8; i++) begin
            #10;
            $display("Cycle %d: Serial_out = %b, Done = %b, Count = %d, Shift_reg = %8b", 
                     i+1, serial_out, done, DUT.count, DUT.shift_reg);
        end
        
        #10;
        display_state("After all bits shifted:");
        
        // Test 3: Load new data while shifting (should restart)
        $display("\n=== TEST 3: Load during shifting (0x55 then 0x33) ===");
        parallel_in = 8'h55;  // 01010101
        load = 1;
        #10;
        display_state("First Data Loaded (0x55):");
        
        load = 0;
        
        // Shift a few bits
        #10;
        $display("After 1 shift: Serial_out = %b, Count = %d, Shift_reg = %8b", 
                 serial_out, DUT.count, DUT.shift_reg);
        #10;
        $display("After 2 shifts: Serial_out = %b, Count = %d, Shift_reg = %8b", 
                 serial_out, DUT.count, DUT.shift_reg);
        
        // Load new data while shifting
        parallel_in = 8'h33;  // 00110011
        load = 1;
        #10;
        display_state("New Data Loaded (0x33) - Should restart:");
        
        load = 0;
        
        // Continue shifting the new data
        for (int i = 0; i < 8; i++) begin
            #10;
            $display("Cycle %d: Serial_out = %b, Done = %b, Count = %d, Shift_reg = %8b", 
                     i+1, serial_out, done, DUT.count, DUT.shift_reg);
        end
        
        // Test 4: Reset during operation
        $display("\n=== TEST 4: Reset during shifting ===");
        parallel_in = 8'hC3;  // 11000011
        load = 1;
        #10;
        display_state("Data Loaded (0xC3):");
        
        load = 0;
        
        // Shift a few bits
        #10;
        #10;
        #10;
        $display("After 3 shifts: Serial_out = %b, Count = %d, Shift_reg = %8b", 
                 serial_out, DUT.count, DUT.shift_reg);
        
        // Reset during operation
        rst_n = 0;
        #10;
        display_state("Reset Applied During Operation:");
        
        rst_n = 1;
        #10;
        display_state("Reset Released:");
        
        $display("\n==========================================");
        $display("Test completed!");
        
        #20;
        $finish;
    end

endmodule
```

Verilator Simulation Output:
Starting Parallel-to-Serial Converter Test

After Reset:
Time:   10 | Load: 0 | Parallel_in: 00000000 (00) | Serial_out: 0 | Done: 0 |
Count: 0 | Shift_reg: 00000000
---------------------------------------------------------
Reset Released:
Time:   20 | Load: 0 | Parallel_in: 00000000 (00) | Serial_out: 0 | Done: 0 |
Count: 1 | Shift_reg: 00000000
---------------------------------------------------------

=== TEST 1: Converting 0xAA (10101010) ===
Data Loaded:
Time:   30 | Load: 1 | Parallel_in: 10101010 (aa) | Serial_out: 1 | Done: 0 |
Count: 0 | Shift_reg: 10101010
---------------------------------------------------------
Cycle           1: Serial_out = 0, Done = 0, Count = 1, Shift_reg = 01010100
Cycle           2: Serial_out = 1, Done = 0, Count = 2, Shift_reg = 10101000
Cycle           3: Serial_out = 0, Done = 0, Count = 3, Shift_reg = 01010000
Cycle           4: Serial_out = 1, Done = 0, Count = 4, Shift_reg = 10100000
Cycle           5: Serial_out = 

0

### Example: Generate Loop for Parameterized Design

In [8]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_7__ripple_carry_adder/"
files = ["ripple_carry_adder.sv", "ripple_carry_adder_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// ripple_carry_adder.sv

// Full adder module (building block)
module full_adder(
    input logic a, b, cin,
    output logic sum, cout
);
    assign sum = a ^ b ^ cin;
    assign cout = (a & b) | (a & cin) | (b & cin);
endmodule

// Ripple carry adder using generate block
module ripple_carry_adder #(parameter WIDTH = 8)(
    input logic [WIDTH-1:0] a, b,
    input logic cin,
    output logic [WIDTH-1:0] sum,
    output logic cout
);
    logic [WIDTH:0] carry;
    
    assign carry[0] = cin;
    
    generate
        for (genvar i = 0; i < WIDTH; i++) begin : adder_stage
            full_adder fa (
                .a(a[i]),
                .b(b[i]),
                .cin(carry[i]),
                .sum(sum[i]),
                .cout(carry[i+1])
            );
        end
    endgenerate
    
    assign cout = carry[WIDTH];
endmodule
```

```systemverilog
// ripple_carry_adder_testbench.sv
module ripple_carry_adder_testbench;
    // Parameters for different width testing
    parameter WIDTH_8 = 8;
    parameter WIDTH_4 = 4;
    
    // Testbench signals for 8-bit adder
    logic [WIDTH_8-1:0] a8, b8, sum8;
    logic cin8, cout8;
    
    // Testbench signals for 4-bit adder
    logic [WIDTH_4-1:0] a4, b4, sum4;
    logic cin4, cout4;
    
    // Expected results
    logic [WIDTH_8:0] expected_result8;
    logic [WIDTH_4:0] expected_result4;
    
    // Instantiate 8-bit ripple carry adder
    ripple_carry_adder #(.WIDTH(WIDTH_8)) DUT_8bit (
        .a(a8),
        .b(b8),
        .cin(cin8),
        .sum(sum8),
        .cout(cout8)
    );
    
    // Instantiate 4-bit ripple carry adder
    ripple_carry_adder #(.WIDTH(WIDTH_4)) DUT_4bit (
        .a(a4),
        .b(b4),
        .cin(cin4),
        .sum(sum4),
        .cout(cout4)
    );
    
    // Task to test 8-bit adder
    task test_8bit_adder(logic [7:0] test_a, logic [7:0] test_b, logic test_cin, string description);
        a8 = test_a;
        b8 = test_b;
        cin8 = test_cin;
        expected_result8 = {1'b0, test_a} + {1'b0, test_b} + {8'b0, test_cin};
        #1; // Wait for combinational logic
        
        $display("8-bit Test: %s", description);
        $display("  A = %8b (%3d), B = %8b (%3d), Cin = %b", a8, a8, b8, b8, cin8);
        $display("  Sum = %8b (%3d), Cout = %b", sum8, sum8, cout8);
        $display("  Expected: %9b (%3d)", expected_result8, expected_result8);
        $display("  Result: %s", ({cout8, sum8} == expected_result8) ? "PASS" : "FAIL");
        $display("  Carry chain: %b", DUT_8bit.carry);
        $display();
    endtask
    
    // Task to test 4-bit adder
    task test_4bit_adder(logic [3:0] test_a, logic [3:0] test_b, logic test_cin, string description);
        a4 = test_a;
        b4 = test_b;
        cin4 = test_cin;
        expected_result4 = {1'b0, test_a} + {1'b0, test_b} + {4'b0, test_cin};
        #1; // Wait for combinational logic
        
        $display("4-bit Test: %s", description);
        $display("  A = %4b (%2d), B = %4b (%2d), Cin = %b", a4, a4, b4, b4, cin4);
        $display("  Sum = %4b (%2d), Cout = %b", sum4, sum4, cout4);
        $display("  Expected: %5b (%2d)", expected_result4, expected_result4);
        $display("  Result: %s", ({cout4, sum4} == expected_result4) ? "PASS" : "FAIL");
        $display("  Carry chain: %b", DUT_4bit.carry);
        $display();
    endtask
    
    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("ripple_carry_adder_testbench.vcd");
        $dumpvars(0, ripple_carry_adder_testbench);
        
        $display("Starting Ripple Carry Adder Test");
        $display("================================");
        $display();
        
        // === 8-BIT ADDER TESTS ===
        $display("=== 8-BIT RIPPLE CARRY ADDER TESTS ===");
        $display();
        
        // Basic addition tests
        test_8bit_adder(8'd0, 8'd0, 1'b0, "Zero + Zero");
        test_8bit_adder(8'd15, 8'd10, 1'b0, "15 + 10");
        test_8bit_adder(8'd255, 8'd0, 1'b0, "255 + 0");
        test_8bit_adder(8'd128, 8'd127, 1'b0, "128 + 127");
        
        // Test with carry in
        test_8bit_adder(8'd100, 8'd50, 1'b1, "100 + 50 + 1 (with carry in)");
        test_8bit_adder(8'd255, 8'd255, 1'b1, "255 + 255 + 1 (maximum with carry)");
        
        // Overflow tests
        test_8bit_adder(8'd255, 8'd1, 1'b0, "255 + 1 (overflow)");
        test_8bit_adder(8'd200, 8'd100, 1'b0, "200 + 100 (overflow)");
        
        // Pattern tests
        test_8bit_adder(8'b10101010, 8'b01010101, 1'b0, "Alternating patterns");
        test_8bit_adder(8'b11110000, 8'b00001111, 1'b0, "Complementary patterns");
        
        $display("=== 4-BIT RIPPLE CARRY ADDER TESTS ===");
        $display();
        
        // === 4-BIT ADDER TESTS ===
        test_4bit_adder(4'd0, 4'd0, 1'b0, "Zero + Zero");
        test_4bit_adder(4'd7, 4'd8, 1'b0, "7 + 8");
        test_4bit_adder(4'd15, 4'd0, 1'b0, "15 + 0");
        test_4bit_adder(4'd9, 4'd6, 1'b1, "9 + 6 + 1 (with carry in)");
        test_4bit_adder(4'd15, 4'd15, 1'b0, "15 + 15 (maximum)");
        test_4bit_adder(4'd15, 4'd15, 1'b1, "15 + 15 + 1 (maximum with carry)");
        
        // Overflow tests
        test_4bit_adder(4'd15, 4'd1, 1'b0, "15 + 1 (overflow)");
        test_4bit_adder(4'd10, 4'd8, 1'b0, "10 + 8 (overflow)");
        
        // === EXHAUSTIVE 4-BIT TEST ===
        $display("=== EXHAUSTIVE 4-BIT TEST (selected cases) ===");
        $display();
        
        // Test a few representative cases from exhaustive testing
        for (int i = 0; i < 16; i += 5) begin
            for (int j = 0; j < 16; j += 7) begin
                for (int c = 0; c < 2; c++) begin
                    test_4bit_adder(i[3:0], j[3:0], c[0], $sformatf("Exhaustive: %d + %d + %d", i, j, c));
                end
            end
        end
        
        // === TIMING TEST ===
        $display("=== PROPAGATION DELAY TEST ===");
        $display();
        
        // Test carry propagation through all stages
        a8 = 8'b11111111;
        b8 = 8'b00000000;
        cin8 = 1'b1;
        $display("Testing carry propagation: 11111111 + 00000000 + 1");
        $display("This should cause carry to ripple through all stages");
        
        #1;
        $display("Final result: Sum = %8b, Cout = %b", sum8, cout8);
        $display("Carry chain: %b", DUT_8bit.carry);
        $display();
        
        $display("================================");
        $display("All tests completed!");
        $display("================================");
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:
Starting Ripple Carry Adder Test

=== 8-BIT RIPPLE CARRY ADDER TESTS ===

8-bit Test: Zero + Zero
  A = 00000000 (  0), B = 00000000 (  0), Cin = 0
  Sum = 00000000 (  0), Cout = 0
  Expected: 000000000 (  0)
  Result: PASS
  Carry chain: 000000000

8-bit Test: 15 + 10
  A = 00001111 ( 15), B = 00001010 ( 10), Cin = 0
  Sum = 00011001 ( 25), Cout = 0
  Expected: 000011001 ( 25)
  Result: PASS
  Carry chain: 000011100

8-bit Test: 255 + 0
  A = 11111111 (255), B = 00000000 (  0), Cin = 0
  Sum = 11111111 (255), Cout = 0
  Expected: 011111111 (255)
  Result: PASS
  Carry chain: 000000000

8-bit Test: 128 + 127
  A = 10000000 (128), B = 01111111 (127), Cin = 0
  Sum = 11111111 (255), Cout = 0
  Expected: 011111111 (255)
  Result: PASS
  Carry chain: 000000000

8-bit Test: 100 + 50 + 1 (with carry in)
  A = 01100100 (100), B = 00110010 ( 50), Cin = 1
  Sum = 10010111 (151), Cout = 0
  Expected: 010010111 (151)
  Result: PASS
  Carry chain: 011000001

8-bit Test

0

## while Loop

The `while` loop continues as long as the condition is true.

```systemverilog
while (condition) begin
    // statements
end
```

### Example 8: Testbench with while Loop

In [9]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_8__counter_4bit/"
files = ["counter_4bit.sv", "counter_4bit_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// counter_4bit.sv
module counter_4bit (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       enable,
    output logic [3:0] count
);

    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            count <= 4'b0000;
            $display("Counter reset - count = %0d", count);
        end
        else if (enable) begin
            count <= count + 1;
            $display("Counter enabled - count = %0d", count + 1);
        end
        else begin
            $display("Counter disabled - count = %0d", count);
        end
    end

endmodule
```

```systemverilog
// counter_4bit_testbench.sv
module counter_4bit_testbench;
    logic clk, rst_n, enable;
    logic [3:0] count;
    integer test_cycles;
    
    // Instantiate the 4-bit counter design under test
    counter_4bit dut_counter (
        .clk(clk),
        .rst_n(rst_n),
        .enable(enable),
        .count(count)
    );
    
    // Clock generation - 10ns period (100MHz)
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Main test sequence
    initial begin
        // Setup waveform dumping
        $dumpfile("counter_4bit_testbench.vcd");
        $dumpvars(0, counter_4bit_testbench);
        
        // Initialize test signals
        rst_n = 0;
        enable = 0;
        test_cycles = 0;
        
        $display("=== 4-bit Counter Testbench Started ===");
        $display("Time: %0t", $time);
        $display();
        
        // Apply reset for 10ns
        #10 rst_n = 1;
        $display("Reset deasserted at time %0t", $time);
        
        // Enable counting after reset
        #10 enable = 1;
        $display("Counter enabled at time %0t", $time);
        $display();
        
        // Run test for 20 clock cycles
        while (test_cycles < 20) begin
            @(posedge clk);
            $display("Cycle %2d: count = %2d (0x%h) at time %0t", 
                     test_cycles, count, count, $time);
            test_cycles++;
            
            // Test disable functionality at cycle 10
            if (test_cycles == 10) begin
                enable = 0;
                $display(">>> Counter disabled at cycle %0d <<<", test_cycles);
            end
            
            // Re-enable at cycle 15
            if (test_cycles == 15) begin
                enable = 1;
                $display(">>> Counter re-enabled at cycle %0d <<<", test_cycles);
            end
        end
        
        $display();
        $display("=== Testbench Completed Successfully ===");
        $display("Final count value: %0d", count);
        $display("Total simulation time: %0t", $time);
        $finish;
    end
    
    // Overflow detection monitor
    always @(posedge clk) begin
        if (rst_n && enable && count == 4'b1111) begin
            $display("*** OVERFLOW WARNING: Counter reached maximum value (15) ***");
        end
    end
    
    // Value change monitor for debugging
    always @(count) begin
        if (rst_n) begin
            $display("Count changed to: %0d", count);
        end
    end

endmodule
```

Verilator Simulation Output:
=== 4-bit Counter Testbench Started ===
Time: 0

Counter reset - count = 0
Reset deasserted at time 10
Counter disabled - count = 0
Counter enabled at time 20

Cycle  0: count =  0 (0x0) at time 25
Counter enabled - count = 1
Count changed to: 1
Cycle  1: count =  1 (0x1) at time 35
Counter enabled - count = 2
Count changed to: 2
Cycle  2: count =  2 (0x2) at time 45
Counter enabled - count = 3
Count changed to: 3
Cycle  3: count =  3 (0x3) at time 55
Counter enabled - count = 4
Count changed to: 4
Cycle  4: count =  4 (0x4) at time 65
Counter enabled - count = 5
Count changed to: 5
Cycle  5: count =  5 (0x5) at time 75
Counter enabled - count = 6
Count changed to: 6
Cycle  6: count =  6 (0x6) at time 85
Counter enabled - count = 7
Count changed to: 7
Cycle  7: count =  7 (0x7) at time 95
Counter enabled - count = 8
Count changed to: 8
Cycle  8: count =  8 (0x8) at time 105
Counter enabled - count = 9
Count changed to: 9
Cycle  9: count =  9 (0x9) at time 1

0

## do-while Loop

The `do-while` loop executes at least once before checking the condition.

```systemverilog
do begin
    // statements
end while (condition);
```

### Example 9: Random Test Generation

In [10]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_9__unique_random_generator/"
files = ["unique_random_generator.sv", "unique_random_generator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// unique_random_generator.sv
module unique_random_generator ();               // Design under test
  
  // Test class that generates unique consecutive random values
  class random_test;
    rand bit [7:0] data;
    bit [7:0] prev_value;
    
    // Function to generate values different from previous one
    function void generate_unique_values();
      int success;
      do begin
        success = randomize();
        if (success == 0) begin
          $error("Randomization failed!");
          break;
        end
      end while (data == prev_value);
      prev_value = data;
    endfunction
  endclass

  // Design logic with random test functionality
  initial begin
    random_test rnd_gen;
    bit [7:0] value_history[5];
    
    $display();                               // Display empty line
    $display("Hello from design!");          // Display message
    $display("=== Unique Random Value Generation ===");
    
    // Create instance of random test class
    rnd_gen = new();
    
    // Generate and display 5 unique consecutive values
    for (int i = 0; i < 5; i++) begin
      rnd_gen.generate_unique_values();
      value_history[i] = rnd_gen.data;
      
      $display("Generation %0d: Value=0x%02h, Previous=0x%02h", 
               i+1, 
               rnd_gen.data, 
               rnd_gen.prev_value);
    end
    
    $display("Design random generation completed!");
  end

endmodule
```

```systemverilog
// unique_random_generator_testbench.sv
module unique_random_generator_testbench;  // Testbench module
  unique_random_generator UNIQUE_RANDOM_GENERATOR();  // Instantiate design under test

  // Additional testbench-specific random testing
  class random_test;
    rand bit [7:0] data;
    bit [7:0] prev_value;
    
    function void generate_unique_values();
      int success;
      do begin
        success = randomize();
        if (success == 0) begin
          $error("Randomization failed!");
          break;
        end
      end while (data == prev_value);
      prev_value = data;
    endfunction
  endclass

  initial begin
    random_test tb_rnd_gen;
    
    // Dump waves
    $dumpfile("unique_random_generator_testbench.vcd");       // Specify the VCD file
    $dumpvars(0, unique_random_generator_testbench);          // Dump all variables in the test module
    #1;                                       // Wait for a time unit
    $display("Hello from testbench!");        // Display message
    $display();                               // Display empty line
    
    // Testbench-specific random value testing
    $display("=== Testbench Random Value Verification ===");
    tb_rnd_gen = new();
    
    // Test multiple generations to verify uniqueness
    for (int i = 0; i < 8; i++) begin
      tb_rnd_gen.generate_unique_values();
      $display("TB Test %0d: Generated=0x%02h, Previous=0x%02h, Unique=%s", 
               i+1,
               tb_rnd_gen.data,
               tb_rnd_gen.prev_value,
               (i == 0) ? "N/A" : "YES");
      #5; // Small delay between generations
    end
    
    $display("Testbench verification completed!");
    #10;
    $finish;
  end

endmodule
```

Verilator Simulation Output:
- Verilator: Walltime 32.008 s (elab=0.003, cvt=0.094, bld=30.907); cpu 0.066 s
on 1 threads; alloced 20.176 MB

Hello from design!
=== Unique Random Value Generation ===
Generation 1: Value=0xc4, Previous=0xc4
Generation 2: Value=0x9c, Previous=0x9c
Generation 3: Value=0x02, Previous=0x02
Generation 4: Value=0xe4, Previous=0xe4
Generation 5: Value=0x78, Previous=0x78
Design random generation completed!
Hello from testbench!

=== Testbench Random Value Verification ===
TB Test 1: Generated=0x9c, Previous=0x9c, Unique=N/A
TB Test 2: Generated=0x02, Previous=0x02, Unique=YES
TB Test 3: Generated=0xe4, Previous=0xe4, Unique=YES
TB Test 4: Generated=0x78, Previous=0x78, Unique=YES
TB Test 5: Generated=0xbc, Previous=0xbc, Unique=YES
TB Test 6: Generated=0xb6, Previous=0xb6, Unique=YES
TB Test 7: Generated=0xe4, Previous=0xe4, Unique=YES
TB Test 8: Generated=0xb7, Previous=0xb7, Unique=YES
Testbench verification completed!
Process finished with return code: 0
Re

0

## foreach Loops

The `foreach` loop iterates over arrays, providing a clean syntax for array operations.

```systemverilog
foreach (array_name[i]) begin
    // statements using array_name[i]
end
```

### Example 10: Array Processing

In [21]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_10__array_processor/"
files = ["array_processor.sv", "array_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// array_processor.sv
module array_processor ();                    // Design under test
  
  // Array processing class with various operations
  class array_operations;
    logic [7:0] data_array[16];
    logic [15:0] sum;                         // Fixed: Increased to 16-bit to prevent overflow
    logic [7:0] avg;
    logic [7:0] min_val;
    logic [7:0] max_val;
    integer array_size;
    
    function new();
      array_size = 16;
      initialize_array();
    endfunction
    
    // Function to initialize array with pattern
    function void initialize_array();
      foreach (data_array[i]) begin
        data_array[i] = 8'(i * 2 + 1);  // Odd numbers: 1, 3, 5, 7, ...
      end
    endfunction
    
    // Function to calculate sum of array elements
    function void calculate_sum();
      sum = 16'h0000;
      foreach (data_array[i]) begin
        sum = sum + 16'(data_array[i]);  // Fixed: Explicit cast and assignment to avoid width warnings
      end
    endfunction
    
    // Function to calculate average
    function void calculate_average();
      calculate_sum();
      avg = 8'(sum / array_size);             // Fixed: Now uses correct sum value
    endfunction
    
    // Function to find minimum and maximum values
    function void find_min_max();
      min_val = data_array[0];
      max_val = data_array[0];
      
      foreach (data_array[i]) begin
        if (data_array[i] < min_val) min_val = data_array[i];
        if (data_array[i] > max_val) max_val = data_array[i];
      end
    endfunction
    
    // Function to display array contents
    function void display_array();
      $display("Array contents:");
      foreach (data_array[i]) begin
        $display("  data_array[%0d] = %0d (0x%02h)", i, data_array[i], data_array[i]);
      end
    endfunction
    
    // Function to display statistics
    function void display_statistics();
      $display("Array Statistics:");
      $display("  Size: %0d elements", array_size);
      $display("  Sum:  %0d", sum);
      $display("  Avg:  %0d", avg);
      $display("  Min:  %0d", min_val);
      $display("  Max:  %0d", max_val);
    endfunction
  endclass

  // Design logic with array processing functionality
  initial begin
    array_operations arr_proc;
    
    $display();                               // Display empty line
    $display("Hello from design!");          // Display message
    $display("=== Array Processing Operations ===");
    
    // Create instance of array operations class
    arr_proc = new();
    
    // Display initial array
    arr_proc.display_array();
    $display();
    
    // Perform array operations
    arr_proc.calculate_sum();
    arr_proc.calculate_average();
    arr_proc.find_min_max();
    
    // Display results
    arr_proc.display_statistics();
    
    $display("Design array processing completed!");
  end

endmodule
```

```systemverilog
// array_processor_testbench.sv
module array_processor_testbench;             // Testbench module
  array_processor ARRAY_PROCESSOR();          // Instantiate design under test

  // Extended testbench class for additional array testing
  class array_test;
    logic [7:0] test_array[16];
    logic [15:0] expected_sum;                // Fixed: Increased to 16-bit to prevent overflow
    logic [15:0] actual_sum;                  // Fixed: Increased to 16-bit to prevent overflow
    integer test_size;
    
    function new();
      test_size = 16;
    endfunction
    
    // Function to initialize test array with different pattern
    function void initialize_test_array();
      foreach (test_array[i]) begin
        // Fixed: Proper casting and clearer logic
        if (i % 2 == 0) begin
          test_array[i] = 8'(i * 3);         // Even indices: 0, 6, 12, 18, ...
        end else begin
          test_array[i] = 8'(i + 10);        // Odd indices: 11, 12, 13, 14, ...
        end
      end
    endfunction
    
    // Function to calculate expected sum
    function void calculate_expected_sum();
      expected_sum = 16'h0000;
      foreach (test_array[i]) begin
        expected_sum = expected_sum + 16'(test_array[i]);  // Fixed: Explicit cast and assignment
      end
    endfunction
    
    // Function to verify array operations
    function bit verify_sum();
      actual_sum = 16'h0000;
      foreach (test_array[i]) begin
        actual_sum = actual_sum + 16'(test_array[i]);    // Fixed: Explicit cast and assignment
      end
      return (actual_sum == expected_sum);
    endfunction
    
    // Function to display test array
    function void display_test_array();
      $display("Test Array Pattern:");
      foreach (test_array[i]) begin
        $display("  test_array[%0d] = %0d (0x%02h)", i, test_array[i], test_array[i]);
      end
    endfunction
    
    // Function to run verification
    function void run_verification();
      bit sum_check;
      
      // Array is already initialized, just calculate and verify
      calculate_expected_sum();
      sum_check = verify_sum();
      
      $display("=== Verification Results ===");
      $display("Expected Sum: %0d", expected_sum);
      $display("Actual Sum:   %0d", actual_sum);
      $display("Sum Check:    %s", sum_check ? "PASS" : "FAIL");
      
      if (sum_check) begin
        $display("Array sum verification PASSED");
      end else begin
        $display("Array sum verification FAILED");
      end
    endfunction
  endclass

  initial begin
    array_test tb_arr_test;
    
    // Dump waves
    $dumpfile("array_processor_testbench.vcd");    // Specify the VCD file
    $dumpvars(0, array_processor_testbench);       // Dump all variables in the test module
    #10;                                            // Wait for 10 time units
    $display("Hello from testbench!");             // Display message
    $display();                                    // Display empty line
    
    // Testbench-specific array testing
    $display("=== Testbench Array Verification ===");
    tb_arr_test = new();

    // Initialize test array first, then display
    tb_arr_test.initialize_test_array();
    tb_arr_test.display_test_array();
    $display();
    
    // Run verification tests
    tb_arr_test.run_verification();
    
    $display();
    $display("=== Additional Array Tests ===");
    
    // Test with different array sizes conceptually
    for (int test_case = 1; test_case <= 3; test_case++) begin
      $display("Test Case %0d: Running array operations...", test_case);
      
      // Simulate different processing scenarios
      case (test_case)
        1: $display("  Processing sequential data pattern");
        2: $display("  Processing alternating data pattern");  
        3: $display("  Processing random-like data pattern");
      endcase
      
      #5; // Small delay between test cases
    end
    
    $display("Testbench verification completed!");
    #10;
    $display();
    $finish;
  end

endmodule
```

Verilator Simulation Output:

Hello from design!
=== Array Processing Operations ===
Array contents:
  data_array[0] = 1 (0x01)
  data_array[1] = 3 (0x03)
  data_array[2] = 5 (0x05)
  data_array[3] = 7 (0x07)
  data_array[4] = 9 (0x09)
  data_array[5] = 11 (0x0b)
  data_array[6] = 13 (0x0d)
  data_array[7] = 15 (0x0f)
  data_array[8] = 17 (0x11)
  data_array[9] = 19 (0x13)
  data_array[10] = 21 (0x15)
  data_array[11] = 23 (0x17)
  data_array[12] = 25 (0x19)
  data_array[13] = 27 (0x1b)
  data_array[14] = 29 (0x1d)
  data_array[15] = 31 (0x1f)

Array Statistics:
  Size: 16 elements
  Sum:  256
  Avg:  16
  Min:  1
  Max:  31
Design array processing completed!
Hello from testbench!

=== Testbench Array Verification ===
Test Array Pattern:
  test_array[0] = 0 (0x00)
  test_array[1] = 11 (0x0b)
  test_array[2] = 6 (0x06)
  test_array[3] = 13 (0x0d)
  test_array[4] = 12 (0x0c)
  test_array[5] = 15 (0x0f)
  test_array[6] = 18 (0x12)
  test_array[7] = 17 (0x11)
  test_array[8] = 24 (0x18)
  

0

### Example 11: Multi-dimensional Array

In [12]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_11__matrix_processor/"
files = ["matrix_processor.sv", "matrix_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// matrix_processor.sv
module matrix_processor ();               // Matrix operations design under test
    logic [7:0] matrix[4][4];
    logic [7:0] row_sum[4];
    
    initial begin
        $display();
        $display("Matrix Processor: Starting operations...");
        
        // Initialize matrix
        foreach (matrix[i]) begin
            foreach (matrix[i][j]) begin
                matrix[i][j] = 8'(i + j);
            end
        end
        
        // Calculate row sums
        foreach (row_sum[i]) begin
            row_sum[i] = 0;
            foreach (matrix[i][j]) begin
                row_sum[i] += matrix[i][j];
            end
        end
        
        // Display results
        $display("Matrix values and row sums:");
        foreach (row_sum[i]) begin
            $display("Row %0d sum = %0d", i, row_sum[i]);
        end
        
        $display("Matrix Processor: Operations completed.");
    end
endmodule
```

```systemverilog
// matrix_processor_testbench.sv
module matrix_processor_testbench;  // Testbench module
  matrix_processor MATRIX_DUT();    // Instantiate matrix processor design under test

  initial begin
    // Dump waves
    $dumpfile("matrix_processor_testbench.vcd");    // Specify the VCD file
    $dumpvars(0, matrix_processor_testbench);       // Dump all variables in the test module
    #1;                                             // Wait for a time unit

    $display("Testbench: Starting matrix processor validation...");
    $display();                                     // Display empty line
    
    // Wait for design to complete
    #10;
    
    $display("Testbench: Matrix processor validation completed!");
  end

endmodule
```

Verilator Simulation Output:

Matrix Processor: Starting operations...
Matrix values and row sums:
Row 0 sum = 6
Row 1 sum = 10
Row 2 sum = 14
Row 3 sum = 18
Matrix Processor: Operations completed.
Testbench: Starting matrix processor validation...

Process finished with return code: 0
Removing Chapter_4_examples/example_11__matrix_processor/obj_dir directory...
Chapter_4_examples/example_11__matrix_processor/obj_dir removed successfully.


0

## repeat Statements

The `repeat` statement executes a block a specified number of times.

```systemverilog
repeat (expression) begin
    // statements
end
```

### Example 12: Clock Generation

In [13]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_12__clock_generator/"
files = ["clock_generator.sv", "clock_generator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// clock_generator.sv
module clock_generator ();               // Clock generator design under test
    logic clk;
    
    initial begin
        $display("Clock Generator: Initializing clock signal...");
        clk = 0;
        
        $display("Clock Generator: Starting 100 clock cycles...");
        repeat (100) begin
            #5 clk = ~clk;
            #5 clk = ~clk;
        end
        
        $display("Clock Generator: Generated 100 clock cycles");
        $display("Clock Generator: Simulation completed.");
        $finish;
    end
endmodule
```

```systemverilog
// clock_generator_testbench.sv
module clock_generator_testbench;  // Testbench module
  clock_generator CLK_GEN_DUT();   // Instantiate clock generator design under test

  initial begin
    // Dump waves
    $dumpfile("clock_generator_testbench.vcd");     // Specify the VCD file
    $dumpvars(0, clock_generator_testbench);        // Dump all variables in the test module
    #1;                                             // Wait for a time unit
    $display("Testbench: Starting clock generator validation...");
    
    // Monitor clock transitions
    $monitor("Time: %0t, Clock: %b", $time, CLK_GEN_DUT.clk);
    
    $display();                                     // Display empty line
    $display("Testbench: Clock generator validation initiated!");
    $display();                                     // Display empty line
  end

endmodule
```

Verilator Simulation Output:
Clock Generator: Initializing clock signal...
Clock Generator: Starting 100 clock cycles...
Testbench: Starting clock generator validation...

Testbench: Clock generator validation initiated!

Time: 5, Clock: 1
Time: 10, Clock: 0
Time: 15, Clock: 1
Time: 20, Clock: 0
Time: 25, Clock: 1
Time: 30, Clock: 0
Time: 35, Clock: 1
Time: 40, Clock: 0
Time: 45, Clock: 1
Time: 50, Clock: 0
Time: 55, Clock: 1
Time: 60, Clock: 0
Time: 65, Clock: 1
Time: 70, Clock: 0
Time: 75, Clock: 1
Time: 80, Clock: 0
Time: 85, Clock: 1
Time: 90, Clock: 0
Time: 95, Clock: 1
Time: 100, Clock: 0
Time: 105, Clock: 1
Time: 110, Clock: 0
Time: 115, Clock: 1
Time: 120, Clock: 0
Time: 125, Clock: 1
Time: 130, Clock: 0
Time: 135, Clock: 1
Time: 140, Clock: 0
Time: 145, Clock: 1
Time: 150, Clock: 0
Time: 155, Clock: 1
Time: 160, Clock: 0
Time: 165, Clock: 1
Time: 170, Clock: 0
Time: 175, Clock: 1
Time: 180, Clock: 0
Time: 185, Clock: 1
Time: 190, Clock: 0
Time: 195, Clock: 1
Time: 200, Clock: 

0

### Example 13: Shift Register Test

In [14]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_13__shift_register/"
files = ["shift_register.sv", "shift_register_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// shift_register.sv
module shift_register (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       serial_in,
    output logic [7:0] parallel_out
);

    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            parallel_out <= 8'h00;
            $display("Shift Register: Reset - parallel_out cleared");
        end else begin
            parallel_out <= {parallel_out[6:0], serial_in};
            $display("Shift Register: Shifted in %b, parallel_out = %b", serial_in, {parallel_out[6:0], serial_in});
        end
    end

endmodule
```

```systemverilog
// shift_register_testbench.sv
module shift_register_testbench;    // Testbench module
    logic clk, rst_n, serial_in;
    logic [7:0] parallel_out;
    logic [7:0] test_pattern = 8'b10110011;
    
    shift_register SHIFT_REG_DUT (.*);  // Instantiate shift register design under test
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    // Main test sequence
    initial begin
        // Dump waves
        $dumpfile("shift_register_testbench.vcd");      // Specify the VCD file
        $dumpvars(0, shift_register_testbench);         // Dump all variables in the test module
        
        $display();                                     // Display empty line
        $display("Testbench: Starting shift register validation...");
        $display();                                     // Display empty line
        
        rst_n = 0;
        serial_in = 0;
        
        #10 rst_n = 1;
        $display("Testbench: Reset released, starting shift operations...");
        
        // Shift in test pattern 10110011
        for (int i = 7; i >= 0; i--) begin
            @(posedge clk);
            serial_in = test_pattern[i];
        end
        
        @(posedge clk);
        $display();                                     // Display empty line
        $display("Testbench: Final parallel output: %b", parallel_out);
        $display("Testbench: Shift register validation completed!");
        $display();                                     // Display empty line
        $finish;
    end

endmodule
```

Verilator Simulation Output:

Testbench: Starting shift register validation...

Shift Register: Reset - parallel_out cleared
Testbench: Reset released, starting shift operations...
Shift Register: Shifted in 1, parallel_out = 00000001
Shift Register: Shifted in 0, parallel_out = 00000010
Shift Register: Shifted in 1, parallel_out = 00000101
Shift Register: Shifted in 1, parallel_out = 00001011
Shift Register: Shifted in 0, parallel_out = 00010110
Shift Register: Shifted in 0, parallel_out = 00101100
Shift Register: Shifted in 1, parallel_out = 01011001
Shift Register: Shifted in 1, parallel_out = 10110011

Testbench: Final parallel output: 10110011
Testbench: Shift register validation completed!

- shift_register_testbench.sv:42: Verilog $finish
Process finished with return code: 0
Removing Chapter_4_examples/example_13__shift_register/obj_dir directory...
Chapter_4_examples/example_13__shift_register/obj_dir removed successfully.


0

## break and continue Statements

SystemVerilog supports `break` and `continue` statements for loop control.

### break Statement

The `break` statement exits the innermost loop immediately.

```systemverilog
for (int i = 0; i < 100; i++) begin
    if (error_condition)
        break;
    // normal processing
end
```

### continue Statement

The `continue` statement skips the rest of the current iteration and continues with the next iteration.

```systemverilog
for (int i = 0; i < 100; i++) begin
    if (skip_condition)
        continue;
    // processing for valid iterations
end
```

### Example 14: Data Validation Loop

In [15]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_14__data_validator/"
files = ["data_validator.sv", "data_validator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_validator.sv
module data_validator ();               // Data validation processor design under test
    logic [7:0] data_stream[100];
    logic [7:0] valid_data[$];
    
    initial begin
        $display("Data Validator: Starting data validation process...");
        
        // Initialize test data with fixed seed approach
        for (int i = 0; i < 100; i++) begin
            data_stream[i] = 8'((i * 17 + 42) % 256);  // Generate pseudo-random pattern
        end

        $display();
        $display("Data Validator: Test data initialized, beginning validation...");
        
        // Process data with validation
        foreach (data_stream[i]) begin
            // Skip invalid data (value 0 or 255)
            if (data_stream[i] == 0 || data_stream[i] == 255) begin
                $display("Data Validator: Skipping invalid data at index %0d: %0d", 
                        i, data_stream[i]);
                continue;
            end
            
            // Break on error pattern (redundant check since 255 is already filtered above)
            if (data_stream[i] == 8'hFF) begin
                $display("Data Validator: Error pattern detected at index %0d", i);
                break;
            end
            
            // Store valid data
            valid_data.push_back(data_stream[i]);
            $display("Data Validator: Valid data stored at index %0d: %0d", i, data_stream[i]);
        end
        
        $display("Data Validator: Processing completed!");
        $display("Data Validator: Processed %0d valid data items out of 100 total", valid_data.size());
        
        // Display first few valid items for verification
        $display("Data Validator: First 10 valid items:");
        for (int i = 0; i < 10 && i < valid_data.size(); i++) begin
            $display("  valid_data[%0d] = %0d", i, valid_data[i]);
        end

        $display();

    end
endmodule
```

```systemverilog
// data_validator_testbench.sv
module data_validator_testbench;  // Testbench module
  data_validator DATA_VAL_DUT();  // Instantiate data validator design under test

  initial begin
    // Dump waves
    $dumpfile("data_validator_testbench.vcd");      // Specify the VCD file
    $dumpvars(0, data_validator_testbench);         // Dump all variables in the test module
    #1;                                             // Wait for a time unit

    $display();                                     // Display empty line
    $display("Testbench: Starting data validator verification...");
    $display();                                     // Display empty line
    
    // Wait for design to complete processing
    #100;
    
    $display("Testbench: Data validator verification completed!");
    $display("Testbench: Check output for validation results and statistics.");
    $display();                                     // Display empty line
  end

endmodule
```

Verilator Simulation Output:
Data Validator: Starting data validation process...

Data Validator: Test data initialized, beginning validation...
Data Validator: Valid data stored at index 0: 42
Data Validator: Valid data stored at index 1: 59
Data Validator: Valid data stored at index 2: 76
Data Validator: Valid data stored at index 3: 93
Data Validator: Valid data stored at index 4: 110
Data Validator: Valid data stored at index 5: 127
Data Validator: Valid data stored at index 6: 144
Data Validator: Valid data stored at index 7: 161
Data Validator: Valid data stored at index 8: 178
Data Validator: Valid data stored at index 9: 195
Data Validator: Valid data stored at index 10: 212
Data Validator: Valid data stored at index 11: 229
Data Validator: Valid data stored at index 12: 246
Data Validator: Valid data stored at index 13: 7
Data Validator: Valid data stored at index 14: 24
Data Validator: Valid data stored at index 15: 41
Data Validator: Valid data stored at index 16: 58
Data Va

0

### Example 15: Search Algorithm
```systemverilog
function int find_first_match(logic [7:0] array[], logic [7:0] target);
    foreach (array[i]) begin
        if (array[i] == target) begin
            return i;  // Found match, return index
        end
        
        // Skip processing for special values
        if (array[i] == 8'hXX) begin
            continue;
        end
        
        // Additional processing could go here
    end
    
    return -1; // Not found
endfunction
```

In [16]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_15__find_first_match/"
files = ["find_first_match.sv", "find_first_match_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// find_first_match.sv
module array_searcher ();               // Array search processor design under test
    logic [7:0] test_array[50];
    logic [7:0] search_targets[5];
    int search_results[5];
    
    // Function to find first match in array
    function int find_first_match(logic [7:0] array[], logic [7:0] target);
        foreach (array[i]) begin
            if (array[i] == target) begin
                return i;  // Found match, return index
            end
            
            // Skip processing for special values (undefined/don't care)
            if (array[i] === 8'bxxxxxxxx) begin
                $display("Array Searcher: Skipping undefined value at index %0d", i);
                continue;
            end
            
            // Additional processing could go here for logging
            // $display("Array Searcher: Checking index %0d, value %0d", i, array[i]);
        end
        
        return -1; // Not found
    endfunction
    
    initial begin
        $display("Array Searcher: Starting array search demonstration...");
        $display();
        
        // Initialize test array with sample data
        for (int i = 0; i < 50; i++) begin
            if (i == 15 || i == 25) begin
                test_array[i] = 8'bxxxxxxxx;  // Insert undefined values
            end else begin
                test_array[i] = 8'((i * 7 + 13) % 128);  // Generate test pattern
            end
        end
        
        // Define search targets
        search_targets[0] = 8'd20;
        search_targets[1] = 8'd50;
        search_targets[2] = 8'd99;
        search_targets[3] = 8'd0;
        search_targets[4] = 8'd127;
        
        $display("Array Searcher: Test array initialized with 50 elements");
        $display("Array Searcher: Undefined values inserted at indices 15 and 25");
        $display();
        
        // Display first 10 array elements for reference
        $display("Array Searcher: First 10 array elements:");
        for (int i = 0; i < 10; i++) begin
            if (test_array[i] === 8'bxxxxxxxx) begin
                $display("  test_array[%0d] = XX (undefined)", i);
            end else begin
                $display("  test_array[%0d] = %0d", i, test_array[i]);
            end
        end
        $display();
        
        // Perform searches
        $display("Array Searcher: Performing searches...");
        foreach (search_targets[j]) begin
            search_results[j] = find_first_match(test_array, search_targets[j]);
            
            if (search_results[j] >= 0) begin
                $display("Array Searcher: Target %0d found at index %0d", 
                        search_targets[j], search_results[j]);
            end else begin
                $display("Array Searcher: Target %0d not found in array", 
                        search_targets[j]);
            end
        end
        
        $display();
        $display("Array Searcher: Search operations completed!");
        
        // Summary of results
        $display("Array Searcher: Search Results Summary:");
        foreach (search_targets[k]) begin
            $display("  Target %0d: %s", 
                    search_targets[k], 
                    (search_results[k] >= 0) ? $sformatf("Found at index %0d", search_results[k]) : "Not found");
        end
        
        $display();
    end
endmodule
```

```systemverilog
// find_first_match_testbench.sv
module find_first_match_testbench;  // Testbench module
  array_searcher SEARCH_DUT();  // Instantiate array searcher design under test

  initial begin
    // Dump waves
    $dumpfile("find_first_match_testbench.vcd");      // Specify the VCD file
    $dumpvars(0, find_first_match_testbench);         // Dump all variables in the test module
    #1;                                               // Wait for a time unit

    $display();                                       // Display empty line
    $display("Testbench: Starting array search function verification...");
    $display();                                       // Display empty line
    
    // Wait for design to complete processing
    #150;
    
    $display("Testbench: Array search function verification completed!");
    $display("Testbench: Check output for search results and function behavior.");
    $display();                                       // Display empty line
    
    // End simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Array Searcher: Starting array search demonstration...

Array Searcher: Test array initialized with 50 elements
Array Searcher: Undefined values inserted at indices 15 and 25

Array Searcher: First 10 array elements:
  test_array[0] = 13
  test_array[1] = 20
  test_array[2] = 27
  test_array[3] = 34
  test_array[4] = 41
  test_array[5] = 48
  test_array[6] = 55
  test_array[7] = 62
  test_array[8] = 69
  test_array[9] = 76

Array Searcher: Performing searches...
Array Searcher: Target 20 found at index 1
Array Searcher: Target 50 not found in array
Array Searcher: Target 99 not found in array
Array Searcher: Target 0 found at index 15
Array Searcher: Target 127 not found in array

Array Searcher: Search operations completed!
Array Searcher: Search Results Summary:
  Target 20: Found at index 1
  Target 50: Not found
  Target 99: Not found
  Target 0: Found at index 15
  Target 127: Not found


Testbench: Starting array search function verification...

Testb

0